# Импорты и загрузки

In [30]:
!pip install midi2audio mido >> None
!apt install fluidsynth >> None
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [22]:
!pip install gdown music21 pretty_midi pydub >> None
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (2.1.1-2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [23]:
import gdown
import argparse
import sys
import os
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
import tensorflow.compat.v1 as v1
from __future__ import division

import pickle
from music21 import stream, note, duration
import mido
import pretty_midi

import shutil
from contextlib import contextmanager

tf.compat.v1.disable_eager_execution()

In [24]:
!git clone https://github.com/ChinghuaChuan/tonnetzDL.git

Cloning into 'tonnetzDL'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), 26.83 KiB | 1.79 MiB/s, done.


In [25]:
url1 = 'https://drive.google.com/uc?id=1g_sxEKwrP9Wqt3YH99eEO1aXmmAqDbui'
url2 = 'https://drive.google.com/u/0/uc?id=1wIT30ESBiKRui8qCsxTTBpNeYkK-Ti90&export=download'
output1 = '/kaggle/working/trained_model.zip'  # Specify the desired output path and filename
output2 = '/kaggle/working/pickle_files.zip' 
gdown.download(url1, output1, quiet=False)
gdown.download(url2, output2, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1g_sxEKwrP9Wqt3YH99eEO1aXmmAqDbui
From (redirected): https://drive.google.com/uc?id=1g_sxEKwrP9Wqt3YH99eEO1aXmmAqDbui&confirm=t&uuid=33ed5444-d70c-449f-ac28-f8ee9e2b5d8a
To: /kaggle/working/trained_model.zip
100%|██████████| 37.1M/37.1M [00:00<00:00, 153MB/s] 
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1wIT30ESBiKRui8qCsxTTBpNeYkK-Ti90&export=download
From (redirected): https://drive.google.com/uc?id=1wIT30ESBiKRui8qCsxTTBpNeYkK-Ti90&export=download&confirm=t&uuid=bb6e4541-9e15-4694-87b1-b29a91589aa2
To: /kaggle/working/pickle_files.zip
100%|██████████| 8.38M/8.38M [00:00<00:00, 98.3MB/s]


'/kaggle/working/pickle_files.zip'

In [26]:
!unzip /kaggle/working/pickle_files.zip
!unzip /kaggle/working/trained_model.zip

%rm /kaggle/working/pickle_files.zip
%rm /kaggle/working/trained_model.zip

Archive:  /kaggle/working/pickle_files.zip
   creating: pickle_files/
  inflating: pickle_files/tonnetz_template_row.txt  
   creating: __MACOSX/
   creating: __MACOSX/pickle_files/
  inflating: __MACOSX/pickle_files/._tonnetz_template_row.txt  
  inflating: pickle_files/MuseData_train_tonnetz_p2.pickle  
  inflating: __MACOSX/pickle_files/._MuseData_train_tonnetz_p2.pickle  
  inflating: pickle_files/MuseData_valid_tonnetz_vocab_p2.pickle  
  inflating: __MACOSX/pickle_files/._MuseData_valid_tonnetz_vocab_p2.pickle  
  inflating: pickle_files/MuseData_valid_tonnetz_p2.pickle  
  inflating: __MACOSX/pickle_files/._MuseData_valid_tonnetz_p2.pickle  
  inflating: pickle_files/MuseData_valid_pianoroll_p2.pickle  
  inflating: __MACOSX/pickle_files/._MuseData_valid_pianoroll_p2.pickle  
  inflating: pickle_files/MuseData_test_tonnetz_p2.pickle  
  inflating: __MACOSX/pickle_files/._MuseData_test_tonnetz_p2.pickle  
  inflating: pickle_files/MuseData_train_pianoroll_p2.pickle  
  inflating:

In [27]:
%cd /kaggle/working/pickle_files
!ls -ahl

/kaggle/working/pickle_files
total 326M
drwxr-xr-x 6 root root 4.0K Jun 14 11:12 .
drwxr-xr-x 7 root root 4.0K Jun 14 11:12 ..
-rwxr-xr-x 1 root root  12M May 15  2018 MuseData_test_pianoroll_p2.pickle
-rw-r--r-- 1 root root  36M May 15  2018 MuseData_test_tonnetz_p2.pickle
-rwxr-xr-x 1 root root  44M May 15  2018 MuseData_train_pianoroll_p2.pickle
-rwxr-xr-x 1 root root 6.9M May 15  2018 MuseData_train_pianoroll_vocab_p2.pickle
-rw-r--r-- 1 root root 137M May 15  2018 MuseData_train_tonnetz_p2.pickle
-rw-r--r-- 1 root root  22M May 15  2018 MuseData_train_tonnetz_vocab_p2.pickle
-rwxr-xr-x 1 root root  15M May 15  2018 MuseData_valid_pianoroll_p2.pickle
-rw-r--r-- 1 root root  47M May 15  2018 MuseData_valid_tonnetz_p2.pickle
-rw-r--r-- 1 root root 9.8M May 15  2018 MuseData_valid_tonnetz_vocab_p2.pickle
-rw-r--r-- 1 root root 3.5K Jun 14 11:12 None
drwxrwxr-x 3 root root 4.0K May 17  2018 __MACOSX
drwxr-xr-x 2 root root 4.0K Jun 14 11:12 newSaves
drwxr-xr-x 2 root root 4.0K May 17  2

# Исправление кода для совместимости

## Train

In [28]:
%%writefile /kaggle/working/tonnenz_dn_v2.py


import tensorflow as tf
import numpy as np
import pickle
import argparse
tf.compat.v1.disable_eager_execution()



parser = argparse.ArgumentParser()
parser.add_argument('vocab', type=str, help='vocabulary pickle file for tonnetz autoencoder (Required)')
parser.add_argument('train', type=str, help='pickle file for training (Required)')
parser.add_argument('--valid', type=str, help='pickle file for validation')
parser.add_argument('--test', type=str, help='pickle file for testing')
parser.add_argument('--CNNepoch', type=int, help='training epoch for CNN autoencoder')
parser.add_argument('--LSTMepoch', type=int, help='training epoch for LSTM')
args = parser.parse_args()

vocab_file = args.vocab
train_file = args.train

if args.valid is not None:
    valid_flag = True
    valid_file = args.valid
else:
    valid_flag = False

if args.test is not None:
    test_flag = True
    test_file = args.test
else:
    test_flag = False

if args.CNNepoch is not None:
    cnn_epoch = args.CNNepoch
else:
    cnn_epoch = 10

if args.LSTMepoch is not None:
    lstm_epoch = args.LSTMepoch
else:
    lstm_epoch = 27

# autoencoder pre-training
def gen_data_cnn():
    l = pickle.load(open(vocab_file, 'rb'))
    return l


def gen_batch_cnn(raw_data, batch_size):
    tonnetz_seq = raw_data
    example = tonnetz_seq[0]
    total_num_examples = len(tonnetz_seq)
    num_batch = total_num_examples // batch_size
    for i in range(num_batch - 1):
        x = tonnetz_seq[batch_size * i:batch_size * (i + 1) - 1]
        yield x


def gen_epochs_cnn(n, batch_size):
    for i in range(n):
        yield gen_batch_cnn(gen_data_cnn(), batch_size + 1)


# LSTM
def gen_data(infile):
    if 'txt' in infile:
        f = open(infile, 'r')
        l = [list(map(float, line.split(','))) for line in f]
    else:
        l = pickle.load(open(infile, 'rb'))
    return l


def gen_batch(raw_data, batch_size, num_steps):
    tonnetz_seq = raw_data
    total_num_examples = len(tonnetz_seq)
    num_seqs = total_num_examples - (num_steps - 1)
    num_batch = num_seqs // batch_size

    for i in range(num_batch - 1):
        batch_x = []
        batch_y = []
        for j in range(batch_size):
            offset = (i * batch_size)
            sequence = tonnetz_seq[offset + j: offset + j + num_steps]
            batch_x = np.append(batch_x, sequence)
            sequence_y = tonnetz_seq[offset + j + num_steps + 1]
            batch_y = np.append(batch_y, sequence_y)
        x = np.reshape(batch_x, (-1, 288))
        y = np.reshape(batch_y, (-1, 288))
        yield x, y


def gen_epochs(n, batch_size, num_steps, infile):
    for i in range(n):
        yield gen_batch(gen_data(infile), batch_size, num_steps)


def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.compat.v1.reset_default_graph()


def conv2d(x, w):
    return tf.nn.conv2d(x, filters=w, strides=[1, 1, 1, 1], padding='SAME')


def max_pool(x, k1, k2):
    return tf.nn.max_pool2d(input=x, ksize=[1, k1, k2, 1], strides=[1, k1, k2, 1], padding='SAME')


# def autoencoder():
reset_graph()

# setting the parameters for both networks
lstm_params = {}
lstm_params['num_steps'] = 16  # number of truncated backprop steps ('n' in the discussion above)
lstm_params['batch_size'] = 500
lstm_params['num_classes'] = 6 * 6 * 10
lstm_params['state_size'] = 6 * 6 * 10

autoencoder_params = {}
autoencoder_params['n_inputs'] = 12 * 24
autoencoder_params['n_hidden1'] = 20
autoencoder_params['n_hidden2'] = 10
autoencoder_params['n_hidden3'] = autoencoder_params['n_hidden1']
autoencoder_params['n_outputs'] = autoencoder_params['n_inputs']
autoencoder_params['learning_rate'] = 1e-4
autoencoder_params['n_epochs'] = cnn_epoch


# building the computational graph
def build_graph(cell_type=None,
                num_weights_for_custom_cell=5,
                state_size=lstm_params['state_size'],
                num_classes=lstm_params['num_classes'],
                batch_size=lstm_params['batch_size'],
                num_steps=lstm_params['num_steps'],
                num_layers=3,
                build_with_dropout=False,
                learning_rate=1e-4,
                autoencoder_params=autoencoder_params):
    initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=2.0, mode=("FAN_AVG").lower(), distribution=("uniform" if True else "truncated_normal"))

    x = tf.compat.v1.placeholder(tf.float32, [None, autoencoder_params['n_inputs']],
                       name='input_placeholder')
    y = tf.compat.v1.placeholder(tf.float32, [None, autoencoder_params['n_inputs']], name="output_placeholder")
    dropout = tf.constant(1.0)
    x_image = tf.reshape(x, shape=[-1, 24, 12, 1])

    weight1 = tf.Variable(initializer([3, 3, 1, autoencoder_params['n_hidden1']]), name='weight1')
    bias1 = tf.Variable(tf.zeros(autoencoder_params['n_hidden1']), name='bias1')
    hidden1 = tf.nn.relu(conv2d(x_image, weight1) + bias1)
    pool1 = max_pool(hidden1, 2, 2)

    weight2 = tf.Variable(initializer([3, 3, autoencoder_params['n_hidden1'], autoencoder_params['n_hidden2']]),
                          name='weight2')
    bias2 = tf.Variable(tf.zeros(autoencoder_params['n_hidden2']), name='bias2')
    hidden2 = tf.nn.relu(conv2d(pool1, weight2) + bias2)
    pool2 = max_pool(hidden2, 2, 1)

    # a fully connected layer for reduced-dimension CNN
    pool2_reshaped = tf.reshape(pool2, shape=[-1, 6 * 6 * autoencoder_params['n_hidden2']])
    final_w = tf.Variable(initializer([6 * 6 * autoencoder_params['n_hidden2'], autoencoder_params['n_outputs']]),
                          name='final_w')
    final_b = tf.Variable(tf.zeros(autoencoder_params['n_outputs']), name='final_b')
    final_o = tf.matmul(pool2_reshaped, final_w) + final_b

    autoencoder_outputs = final_o

    loss_cnn = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=autoencoder_outputs))

    optimizer_cnn = tf.compat.v1.train.AdamOptimizer(autoencoder_params['learning_rate'])
    training_op_cnn = optimizer_cnn.minimize(loss_cnn)

    # LSTM
    fixed_w1 = tf.compat.v1.placeholder(tf.float32, [3, 3, 1, autoencoder_params['n_hidden1']], name='fixed_w1')
    fixed_b1 = tf.compat.v1.placeholder(tf.float32, [autoencoder_params['n_hidden1']], name='fixed_b1')
    fixed_w2 = tf.compat.v1.placeholder(tf.float32, [3, 3, autoencoder_params['n_hidden1'], autoencoder_params['n_hidden2']],
                              name='fixed_w2')
    fixed_b2 = tf.compat.v1.placeholder(tf.float32, [autoencoder_params['n_hidden2']], name='fixed_b2')
    fixed_final_w = tf.compat.v1.placeholder(tf.float32,
                                   [6 * 6 * autoencoder_params['n_hidden2'], autoencoder_params['n_outputs']],
                                   name='fixed_final_w')
    fixed_final_b = tf.compat.v1.placeholder(tf.float32, [autoencoder_params['n_outputs']], name='fixed_final_b')

    lstm_hidden1 = tf.nn.relu(conv2d(x_image, fixed_w1) + fixed_b1)
    lstm_pool1 = max_pool(lstm_hidden1, 2, 2)
    lstm_hidden2 = tf.nn.relu(conv2d(lstm_pool1, fixed_w2) + fixed_b2)
    lstm_pool2 = max_pool(lstm_hidden2, 2, 1)
    lstm_pool2_reshaped = tf.reshape(lstm_pool2, shape=[-1, 6 * 6 * autoencoder_params['n_hidden2']])

    rnn_inputs = tf.reshape(lstm_pool2_reshaped,
                            shape=[-1, lstm_params['num_steps'], 6 * 6 * autoencoder_params['n_hidden2']])

    if cell_type == 'GRU':
        cell = tf.compat.v1.nn.rnn_cell.GRUCell(state_size)
    elif cell_type == 'LSTM':
        cell = tf.compat.v1.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
    elif cell_type == 'LN_LSTM':
        cell = LayerNormalizedLSTMCell(state_size)
    else:
        cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(state_size)

    if build_with_dropout:
        cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=dropout)

    if cell_type == 'LSTM' or cell_type == 'LN_LSTM':
        cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([build_rnn_cell('LSTM', state_size) for _ in range(num_layers)],
                                           state_is_tuple=True)
    else:
        cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([build_rnn_cell('Basic', state_size) for _ in range(num_layers)],
                                           state_is_tuple=True)

    if build_with_dropout:
        cell = tf.contrib.DropoutWrapper(cell, output_keep_prob=dropout)

    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.compat.v1.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)


    with tf.compat.v1.variable_scope('softmax'):
        W = tf.compat.v1.get_variable('W', [state_size, num_classes])
        b = tf.compat.v1.get_variable('b', [num_classes], initializer=tf.compat.v1.constant_initializer(0.0))

    rnn_outputs = rnn_outputs[:, num_steps - 1, :]

    y_image = tf.reshape(y, shape=[-1, 24, 12, 1])
    y_hidden1 = tf.nn.relu(conv2d(y_image, fixed_w1) + fixed_b1)
    y_pool1 = max_pool(y_hidden1, 2, 2)
    y_hidden2 = tf.nn.relu(conv2d(y_pool1, fixed_w2) + fixed_b2)
    y_pool2 = max_pool(y_hidden2, 2, 1)

    y_pool2_reshaped = tf.reshape(y_pool2, shape=[-1, 6 * 6 * autoencoder_params['n_hidden2']])
    rnn_outputs_reshaped = tf.reshape(rnn_outputs, [-1, num_classes])
    logits = tf.matmul(rnn_outputs_reshaped, W) + b

    total_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y_pool2_reshaped))

    # .............
    # decode the output and compare it with original y

    decode_logits = tf.matmul(rnn_outputs_reshaped, fixed_final_w) + fixed_final_b

    tonnetz_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=decode_logits, labels=y))
    valid_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=decode_logits, labels=y),
                                name='valid_loss')
    train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(tonnetz_loss)
    predictions = tf.nn.sigmoid(decode_logits, name='predictions')

    return dict(
        x=x,
        y=y,
        init_state=init_state,
        final_state=final_state,
        total_loss=total_loss,
        train_step=train_step,
        tonnetz_loss=tonnetz_loss,
        valid_loss=valid_loss,
        preds=predictions,
        training_op_cnn=training_op_cnn,
        loss_cnn=loss_cnn,
        autoencoder_outputs=autoencoder_outputs,
        final_w=final_w,
        weight2=weight2,
        weight1=weight1,
        bias1=bias1,
        bias2=bias2,
        final_b=final_b,
        fixed_w1=fixed_w1,
        fixed_b1=fixed_b1,
        fixed_w2=fixed_w2,
        fixed_b2=fixed_b2,
        fixed_final_w=fixed_final_w,
        fixed_final_b=fixed_final_b,
        saver=tf.compat.v1.train.Saver()
    )


def build_rnn_cell(cell_type, state_size):
    if cell_type == 'GRU':
        cell = tf.compat.v1.nn.rnn_cell.GRUCell(state_size)
    elif cell_type == 'LSTM':
        cell = tf.compat.v1.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
    elif cell_type == 'LN_LSTM':
        cell = LayerNormalizedLSTMCell(state_size)
    else:
        cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(state_size)
    return cell


def train_network(g, num_epochs, num_steps, batch_size=32, verbose=True, save=True):
    tf.compat.v1.set_random_seed(2345)
    with tf.compat.v1.Session() as sess:
        with tf.device('/cpu:0'):
            sess.run(tf.compat.v1.global_variables_initializer())

            # pre_training = True
            if (True):
                # pre-training
                print("CNN autoencoder pre-training starts...")
                for idx, batch in enumerate(gen_epochs_cnn(autoencoder_params['n_epochs'], lstm_params['batch_size'])):
                    training_losses = 0
                    print("epoch", idx)
                    step = 0
                    for x_batch in batch:
                        step = step + 1
                        feed_dict = {g['x']: x_batch}
                        _, b_loss, batch_outputs, final_w, weight2, weight1, final_b, bias2, bias1 = sess.run(
                            [g['training_op_cnn'],
                             g['loss_cnn'],
                             g['autoencoder_outputs'],
                             g['final_w'],
                             g['weight2'],
                             g['weight1'],
                             g['final_b'],
                             g['bias2'],
                             g['bias1']],
                            feed_dict)
                        #print("batch", step, ", batch loss:", b_loss)
                        training_losses = training_losses + b_loss
                        if idx == autoencoder_params['n_epochs'] - 1:
                            if step == 1:
                                final_outputs = batch_outputs
                            else:
                                final_outputs = np.append(final_outputs, batch_outputs, axis=0)

                    training_losses = training_losses / step
                    print("epoch average loss:", training_losses)


            # LSTM: using the input from CNN
            training_losses = []
            batch_losses = []
            valid_losses = []
            print("LSTM training starts...")
            for idx, epoch in enumerate(gen_epochs(num_epochs, batch_size, num_steps, train_file)):
                print("epoch", idx)
                sum_tonnetz_loss = 0
                steps = 0
                training_state = None
                for X, Y in epoch:
                    steps += 1

                    feed_dict = {g['x']: X, g['y']: Y, g['fixed_w1']: weight1, g['fixed_b1']: bias1,
                                 g['fixed_w2']: weight2,
                                 g['fixed_b2']: bias2, g['fixed_final_w']: final_w, g['fixed_final_b']: final_b}
                    if training_state is not None:
                        feed_dict[g['init_state']] = training_state
                    training_state, _, tonnetz_loss, final_w, weight2, fixed_final_w = sess.run([g['final_state'],
                                                                                                 g['train_step'],
                                                                                                 g['tonnetz_loss'],
                                                                                                 g['final_w'],
                                                                                                 g['weight2'],
                                                                                                 g['fixed_final_w']],
                                                                                                feed_dict)
                    # training_loss += training_loss_
                    sum_tonnetz_loss += tonnetz_loss
                    print(steps - 1, tonnetz_loss)
                    batch_losses.append(tonnetz_loss)

                if verbose:
                    print("Average training loss for Epoch", idx, ":", sum_tonnetz_loss / steps)
                training_losses.append(sum_tonnetz_loss / steps)

                ##validation after each epoch:
                if valid_flag == True:
                    for idv, epochv in enumerate(gen_epochs(1, batch_size, num_steps, valid_file)):
                        step = 0
                        valid_total_loss =0
                        for xv, yv in epochv:
                            step += 1
                   #        print("validation step:%d" % step)
                            feed_dict = {g['x']: xv, g['y']:yv, g['fixed_w1']:weight1, g['fixed_b1']:bias1, g['fixed_w2']:weight2, g['fixed_b2']:bias2, g['fixed_final_w']:final_w, g['fixed_final_b']:final_b}
                            loss = sess.run(g['valid_loss'], feed_dict)
                   #        print("validation loss: %f" % loss)
                            valid_total_loss += loss
                        valid_losses.append(valid_total_loss/step)
                    print("Average validation loss for Epoch", idx, ":", valid_total_loss / step)

            if test_flag == True:
                for idx, epoch in enumerate(gen_epochs(1, batch_size, num_steps, test_file)):
                    step = 0
                    test_total_loss = 0
                    for x, y in epoch:
                        step += 1

                        feed_dict = {g['x']: x, g['y']: y, g['fixed_w1']: weight1, g['fixed_b1']: bias1,
                                     g['fixed_w2']: weight2, g['fixed_b2']: bias2, g['fixed_final_w']: final_w,
                                     g['fixed_final_b']: final_b}
                        loss = sess.run(g['valid_loss'], feed_dict)
                        print("test step:", step, "with loss", loss)
                        test_total_loss += loss
                    print("Average test loss =", test_total_loss/step)

        if isinstance(save, str):
            g['saver'].save(sess, save)





    return training_losses

g = build_graph(cell_type='LSTM', num_steps=lstm_params['num_steps'])
losses = train_network(g, lstm_epoch, num_steps=lstm_params['num_steps'], batch_size=lstm_params['batch_size'], save="/kaggle/working/save/tonnetz_training_model")



Overwriting /kaggle/working/tonnenz_dn_v2.py


## Test

In [29]:
%%writefile /kaggle/working/tonnenz_generate_v2.py

from __future__ import division
import tensorflow as tf
import numpy as np
import pickle
from numpy import genfromtxt
import argparse
tf.compat.v1.disable_eager_execution()


parser = argparse.ArgumentParser()
parser.add_argument('vocab', type=str, help='vocabulary pickle file for tonnetz autoencoder (Required)')
parser.add_argument('test', type=str, help='pickle file for testing (Required)')
args = parser.parse_args()
vocab_file = args.vocab
test_file = args.test

tonnetz_template = genfromtxt('/kaggle/working/pickle_files/tonnetz_template_row.txt', delimiter=',')

# autoencoder pre-training
def gen_data_cnn():
    l = pickle.load(open(vocab_file, 'rb'))
    return l

def gen_batch_cnn(raw_data, batch_size):
    tonnetz_seq = raw_data
    example = tonnetz_seq[0]
    total_num_examples = len(tonnetz_seq)
    num_batch = total_num_examples // batch_size

    for i in range(num_batch - 1):
        x = tonnetz_seq[batch_size * i:batch_size * (i + 1) - 1]
        yield x


def gen_epochs_cnn(n, batch_size):
    for i in range(n):
        yield gen_batch_cnn(gen_data_cnn(), batch_size+1)

# LSTM
def gen_data(infile):
    l = pickle.load(open(infile, 'rb'))
    return l

def gen_batch(raw_data, batch_size, num_steps):
    tonnetz_seq = raw_data
    example = tonnetz_seq[0]
    total_num_examples = len(tonnetz_seq)
    num_seqs = total_num_examples - (num_steps - 1)
    num_batch = num_seqs // batch_size
    for i in range(num_batch - 1):
        batch_x = []
        batch_y = []

        for j in range(batch_size):
            offset = (i * batch_size)
            sequence = tonnetz_seq[offset + j: offset + j + num_steps]
            batch_x = np.append(batch_x, sequence)
            sequence_y = tonnetz_seq[offset + j + num_steps + 1]
            batch_y = np.append(batch_y, sequence_y)
        x = np.reshape(batch_x, (-1, 288))
        y = np.reshape(batch_y, (-1, 288))
        yield x, y

def gen_epochs(n, batch_size, num_steps, file):
    for i in range(n):
        yield gen_batch(gen_data(file), batch_size, num_steps)

# setting the parameters for both networks
lstm_params = {}
lstm_params['num_steps'] = 16 # number of truncated backprop steps ('n' in the discussion above)
lstm_params['batch_size'] = 500
lstm_params['num_classes'] = 6*6*10
lstm_params['state_size'] = 6*6*10

autoencoder_params = {}
autoencoder_params['n_inputs'] = 12 * 24
autoencoder_params['n_hidden1'] = 20
autoencoder_params['n_hidden2'] = 10
autoencoder_params['n_hidden3'] = autoencoder_params['n_hidden1']
autoencoder_params['n_outputs'] = autoencoder_params['n_inputs']
autoencoder_params['learning_rate'] = 1e-4
autoencoder_params['n_epochs'] = 10

pred_params = {}
pred_params['policy'] = 'majority'
pred_params['threshold'] = 0.3
pred_params['pred_file'] = 'Testpred_' + pred_params['policy'] + str(pred_params['threshold'])+'.pickle'
pred_params['groundtruth_file'] = 'Testgroundtruth_' + pred_params['policy'] + str(pred_params['threshold']) + '.pickle'

def generate_characters(checkpoint):

    with tf.compat.v1.Session() as sess:

        sess.run(tf.compat.v1.global_variables_initializer())
        meta_file = checkpoint + '.meta'
        saver = tf.compat.v1.train.import_meta_graph(meta_file)
        saver.restore(sess, checkpoint)

        print("Restoring variables...")

        graph = tf.compat.v1.get_default_graph()

        # restore the learned weights/biases needed for autoencoder
        x = graph.get_tensor_by_name("input_placeholder:0")  # placeholder
        y = graph.get_tensor_by_name("output_placeholder:0")  # placeholder

        weight1 = sess.run(graph.get_tensor_by_name("weight1:0"))  # a variable
        bias1 = sess.run(graph.get_tensor_by_name("bias1:0"))
        weight2 = sess.run(graph.get_tensor_by_name("weight2:0"))
        bias2 = sess.run(graph.get_tensor_by_name("bias2:0"))
        final_w = sess.run(graph.get_tensor_by_name("final_w:0"))
        final_b = sess.run(graph.get_tensor_by_name("final_b:0"))
        fixed_w1 = graph.get_tensor_by_name("fixed_w1:0")
        fixed_b1 = graph.get_tensor_by_name("fixed_b1:0")
        fixed_w2 = graph.get_tensor_by_name("fixed_w2:0")
        fixed_b2 = graph.get_tensor_by_name("fixed_b2:0")
        fixed_final_w = graph.get_tensor_by_name("fixed_final_w:0")
        fixed_final_b = graph.get_tensor_by_name("fixed_final_b:0")
        predictions = graph.get_tensor_by_name("predictions:0")
        valid_loss = graph.get_tensor_by_name("valid_loss:0")

        out_pred = []
        out_groundtruth = []

        for id, epoch in enumerate(gen_epochs(1, lstm_params['batch_size'], lstm_params['num_steps'], test_file)):
            step = 0
            test_total_loss = 0
            all_predictions = np.empty(shape=[0, 288])

            for xi, yi in epoch:
                step +=1
                feed_dict = {x: xi, y: yi, fixed_w1: weight1, fixed_b1: bias1, fixed_w2: weight2,
                             fixed_b2: bias2, fixed_final_w: final_w, fixed_final_b: final_b}
                pred, loss = sess.run([predictions, valid_loss], feed_dict)
                test_total_loss += loss
                print("batch ", step, " loss = ", loss)
                all_predictions = np.append(all_predictions, pred, axis=0)
                # getting pitch output for the batch
                out_pred.extend(tonnetz2note_batch(pred, pred_params['policy'], pred_params['threshold']))
                out_groundtruth.extend(tonnetz2note_batch(yi, pred_params['policy'], pred_params['threshold']))

            print('The average test loss is: ', test_total_loss/step)

        np.savetxt('/kaggle/working/MuseData_tonnetz_test_predictions.txt', all_predictions)

        print("Writing generated sequences in file:", pred_params['pred_file'])
        with open('/kaggle/working/'+pred_params['pred_file'], 'wb') as f:
            pickle.dump(out_pred, f)

        print("Writing the groundtruth for the generated sequences in file:", pred_params['groundtruth_file'])
        with open('/kaggle/working/'+pred_params['groundtruth_file'], 'wb') as f:
            pickle.dump(out_groundtruth, f)

#policy = 'max'
#threshold = 0.3
pitch_range = [6, 109] # tonnetz pitch range
def tonnetz2note(pred, policy, threshold):
    pitch_out_batch = []
    count = 0
    for frame in pred:
        pitch = pitch_range[0]
        pitch_out = []
        while pitch <= pitch_range[1]:
            index = np.where(tonnetz_template == pitch)
            pitch_preds = []
            for loc in index:
                for i in loc:
                    pitch_preds = np.append(pitch_preds, frame[i])

            pitch_preds = np.array(pitch_preds)
            out = 0
            if policy == 'mean':
                if np.mean(pitch_preds) > threshold:
                    out = 1
            elif policy == 'majority':
                index_count = np.where(pitch_preds > threshold)
                if len(index_count) > (len(pitch_preds)/2):
                    out = 1
            elif policy == 'min':
                if min(pitch_preds) > threshold:
                    out = 1
            else:   # max as default
                if max(pitch_preds) > threshold:
                    out = 1

            if out == 1:
                pitch_out = np.append(pitch_out, pitch)
            pitch +=1

        pitch_out_batch.append(pitch_out)
        count +=1
    return pitch_out_batch


def tonnetz2note_batch(pred_batch, policy, threshold):
    pitch_out_batch = []
    count = 0
    for frame in pred_batch:
        pitch = pitch_range[0]
        pitch_out = []
        while pitch <= pitch_range[1]:
            index = np.where(tonnetz_template == pitch)
            pitch_preds = []
            for loc in index:
                for i in loc:
                    pitch_preds = np.append(pitch_preds, frame[i])

            pitch_preds = np.array(pitch_preds)
            out = 0
            if policy == 'mean':
                if np.mean(pitch_preds) > threshold:
                    out = 1
            elif policy == 'majority':
                index_count = np.where(pitch_preds > threshold)
                if len(index_count[0]) > (len(pitch_preds) / 2):
                    out = 1
            elif policy == 'min':
                if min(pitch_preds) > threshold:
                    out = 1
            else:  # max as default
                if max(pitch_preds) > threshold:
                    out = 1

            if out == 1:
                pitch_out = np.append(pitch_out, pitch)
            pitch += 1

        pitch_out_batch.append(pitch_out)
        count += 1
    return pitch_out_batch


print ("Generating sequences for test file:", test_file)
#generate_characters("./save/tonnetz_training_model")
generate_characters("/kaggle/working/newSaves/MuseData_tonnetz_training_model_epochs27")

Overwriting /kaggle/working/tonnenz_generate_v2.py


In [30]:
%cd /kaggle/working/newSaves
%ls -ahl

/kaggle/working/newSaves
total 38M
drwxr-xr-x 2 root root 4.0K Jun 14 10:25 ./
drwxr-xr-x 7 root root 4.0K Jun 14 11:12 ../
-rw-r--r-- 1 root root  38M Jan 22  2018 MuseData_tonnetz_training_model_epochs27.data-00000-of-00001
-rw-r--r-- 1 root root 1.5K Jan 22  2018 MuseData_tonnetz_training_model_epochs27.index
-rw-r--r-- 1 root root 346K Jan 22  2018 MuseData_tonnetz_training_model_epochs27.meta
-rw-r--r-- 1 root root  137 Jan 22  2018 checkpoint


In [31]:
%cd /kaggle/working/pickle_files
%ls -ahl

/kaggle/working/pickle_files
total 326M
drwxr-xr-x 6 root root 4.0K Jun 14 11:12 ./
drwxr-xr-x 7 root root 4.0K Jun 14 11:12 ../
-rwxr-xr-x 1 root root  12M May 15  2018 MuseData_test_pianoroll_p2.pickle*
-rw-r--r-- 1 root root  36M May 15  2018 MuseData_test_tonnetz_p2.pickle
-rwxr-xr-x 1 root root  44M May 15  2018 MuseData_train_pianoroll_p2.pickle*
-rwxr-xr-x 1 root root 6.9M May 15  2018 MuseData_train_pianoroll_vocab_p2.pickle*
-rw-r--r-- 1 root root 137M May 15  2018 MuseData_train_tonnetz_p2.pickle
-rw-r--r-- 1 root root  22M May 15  2018 MuseData_train_tonnetz_vocab_p2.pickle
-rwxr-xr-x 1 root root  15M May 15  2018 MuseData_valid_pianoroll_p2.pickle*
-rw-r--r-- 1 root root  47M May 15  2018 MuseData_valid_tonnetz_p2.pickle
-rw-r--r-- 1 root root 9.8M May 15  2018 MuseData_valid_tonnetz_vocab_p2.pickle
-rw-r--r-- 1 root root 3.5K Jun 14 11:12 None
drwxrwxr-x 3 root root 4.0K May 17  2018 __MACOSX/
drwxr-xr-x 2 root root 4.0K Jun 14 11:12 newSaves/
drwxr-xr-x 2 root root 4.0K M

In [32]:
templ =(np.array([6,13,8,15,10,17,12,7,14,9,16,11,10,17,12,19,14,21,16,11,18,13,20,15,
        14,21,16,23,18,25,20,15,22,17,24,19,18,25,20,27,22,29,24,19,26,21,28,
        23,22,29,24,31,26,33,28,23,30,25,32,27,26,33,28,35,30,37,32,27,34,29,
        36,31,30,37,32,39,34,41,36,31,38,33,40,35,34,41,36,43,38,45,40,35,42,
        37,44,39,38,45,40,47,42,49,44,39,46,41,48,43,42,49,44,51,46,53,48,43,
        50,45,52,47,46,53,48,55,50,57,52,47,54,49,56,51,50,57,52,59,54,61,56,
        51,58,53,60,55,54,61,56,63,58,65,60,55,62,57,64,59,58,65,60,67,62,69,
        64,59,66,61,68,63,62,69,64,71,66,73,68,63,70,65,72,67,66,73,68,75,70,
        77,72,67,74,69,76,71,70,77,72,79,74,81,76,71,78,73,80,75,74,81,76,83,
        78,85,80,75,82,77,84,79,78,85,80,87,82,89,84,79,86,81,88,83,82,89,84,
        91,86,93,88,83,90,85,92,87,86,93,88,95,90,97,92,87,94,89,96,91,90,97,
        92,99,94,101,96,91,98,93,100,95,94,101,96,103,98,105,100,95,102,97,104,
        99,98,105,100,107,102,109,104,99,106,101,108,103])).reshape(24,12)
print(templ)

[[  6  13   8  15  10  17  12   7  14   9  16  11]
 [ 10  17  12  19  14  21  16  11  18  13  20  15]
 [ 14  21  16  23  18  25  20  15  22  17  24  19]
 [ 18  25  20  27  22  29  24  19  26  21  28  23]
 [ 22  29  24  31  26  33  28  23  30  25  32  27]
 [ 26  33  28  35  30  37  32  27  34  29  36  31]
 [ 30  37  32  39  34  41  36  31  38  33  40  35]
 [ 34  41  36  43  38  45  40  35  42  37  44  39]
 [ 38  45  40  47  42  49  44  39  46  41  48  43]
 [ 42  49  44  51  46  53  48  43  50  45  52  47]
 [ 46  53  48  55  50  57  52  47  54  49  56  51]
 [ 50  57  52  59  54  61  56  51  58  53  60  55]
 [ 54  61  56  63  58  65  60  55  62  57  64  59]
 [ 58  65  60  67  62  69  64  59  66  61  68  63]
 [ 62  69  64  71  66  73  68  63  70  65  72  67]
 [ 66  73  68  75  70  77  72  67  74  69  76  71]
 [ 70  77  72  79  74  81  76  71  78  73  80  75]
 [ 74  81  76  83  78  85  80  75  82  77  84  79]
 [ 78  85  80  87  82  89  84  79  86  81  88  83]
 [ 82  89  84  91  86  93  88  

In [34]:
import numpy as np
import os
import pickle
import pretty_midi
from sklearn.model_selection import train_test_split

def compute_tonnetz(pitch, pitch_matrix):
    tonnetz = np.zeros(288)
    for i in range(pitch_matrix.shape[0]):
        for j in range(pitch_matrix.shape[1]):
            if pitch_matrix[i, j] == pitch:
                tonnetz[i * pitch_matrix.shape[1] + j] = 1
    tonnetz[tonnetz > 1] = 1  # Заменить значения больше 1 на 1
    return tonnetz

def process_midi_file(file_path, pitch_matrix):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    dataset = []
    unique_patterns = []
    for note in midi_data.instruments[0].notes:
        pitch = note.pitch
        start_time = note.start
        end_time = note.end
        beat_duration = midi_data.get_beats(end_time - start_time)
        num_time_steps = int(np.ceil(beat_duration[0]))  # Фиксированное количество интервалов (все интервалы)
        time_step_duration = 1  # Длительность каждого интервала (1 секунда)
        time_step_patterns = None  # Инициализация переменной
        for step in range(num_time_steps):
            step_start_time = start_time + step * time_step_duration
            step_end_time = step_start_time + time_step_duration
            notes_in_step = []
            for note in midi_data.instruments[0].notes:
                if step_start_time <= note.start < step_end_time:
                    notes_in_step.append(note.pitch)
            if notes_in_step:
                tonnetz = np.zeros(288)
                for pitch in notes_in_step:
                    tonnetz += compute_tonnetz(pitch, pitch_matrix)
                tonnetz[tonnetz > 1] = 1  # Заменить значения больше 1 на 1
                if time_step_patterns is None:  # Проверка на инициализацию
                    time_step_patterns = []
                time_step_patterns.append(tonnetz.tolist())
                unique_patterns.append(tonnetz.tolist())
        if time_step_patterns is not None:  # Проверка на инициализацию
            dataset.extend(time_step_patterns)

    return dataset, unique_patterns

# Pattern tonnetz
pitch_matrix = templ

folder_path = "/kaggle/input/jazzset/jazztrimmedfortime"

# Получение списка файлов
midi_files = os.listdir(folder_path)

# Разделение на тренировочный, валидационный и тестовый наборы данных
train_files, temp_files = train_test_split(midi_files, test_size=0.4, random_state=42)
valid_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

train_dataset = []
train_vocab = []
for file_name in train_files:
    file_path = os.path.join(folder_path, file_name)
    dataset, vocab = process_midi_file(file_path, pitch_matrix)
    train_dataset.extend(dataset)
    train_vocab.extend(vocab)

valid_dataset = []
valid_vocab = []
for file_name in valid_files:
    file_path = os.path.join(folder_path, file_name)
    dataset, vocab = process_midi_file(file_path, pitch_matrix)
    valid_dataset.extend(dataset)
    valid_vocab.extend(vocab)

test_dataset = []
test_vocab = []
for file_name in test_files:
    file_path = os.path.join(folder_path, file_name)
    dataset, vocab = process_midi_file(file_path, pitch_matrix)
    test_dataset.extend(dataset)
    test_vocab.extend(vocab)

# Сохранение тренировочных данных в файлы
train_data_path = "/kaggle/working/train_dataset.pickle"
with open(train_data_path, "wb") as f:
    pickle.dump(train_dataset, f)

train_vocab_path = "/kaggle/working/train_vocab.pickle"
with open(train_vocab_path, "wb") as f:
    pickle.dump(train_vocab, f)

# Сохранение валидационных данных в файлы
valid_data_path = "/kaggle/working/valid_dataset.pickle"
with open(valid_data_path, "wb") as f:
    pickle.dump(valid_dataset, f)

valid_vocab_path = "/kaggle/working/valid_vocab.pickle"
with open(valid_vocab_path, "wb") as f:
    pickle.dump(valid_vocab, f)

# Сохранение тестовых данных в файлы
test_data_path = "/kaggle/working/test_dataset.pickle"
with open(test_data_path, "wb") as f:
    pickle.dump(test_dataset, f)

test_vocab_path = "/kaggle/working/test_vocab.pickle"
with open(test_vocab_path, "wb") as f:
    pickle.dump(test_vocab, f)


In [42]:
# Загрузка сгенерированных музыкальных последовательностей из файла
with open('/kaggle/working/test_vocab.pickle', 'rb') as f:
    tvocab = pickle.load(f)

# Загрузка сгенерированных музыкальных последовательностей из файла
with open('/kaggle/working/train_dataset.pickle', 'rb') as f:
    sequences = pickle.load(f)  # Взять все интервалы

print(tvocab[0] == sequences[0])
print(len(tvocab), len(sequences))
print(len(tvocab[0]), len(sequences[1]))

print(sequences[:2])
print(np.count_nonzero(tvocab[0]))

False
66303 190555
288 288
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [37]:
# Код для обучения нейросети

!python /kaggle/working/tonnenz_dn_v2.py \
    --valid /kaggle/working/valid_dataset.pickle \
    --CNNepoch 27 --LSTMepoch 1 \
    /kaggle/working/train_dataset.pickle \
    /kaggle/working/train_vocab.pickle

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [43]:
# генерация

!python /kaggle/working/tonnenz_generate_v2.py \
  /kaggle/working/test_vocab.pickle \
  /kaggle/working/test_dataset.pickle

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [44]:
# Загрузка сгенерированных музыкальных последовательностей из файла
with open('/kaggle/working/Testpred_majority0.3.pickle', 'rb') as f:
    r = pickle.load(f)
filtered_sublists = [sublist for sublist in r if len(sublist) >= 2]
filtered_sublists

[array([57., 60., 62., 64., 69., 72., 76., 81.]),
 array([60., 62., 64., 69., 72., 76., 81.]),
 array([60., 64., 69., 72., 76., 81., 84.]),
 array([64., 69., 72., 76., 81.]),
 array([64., 69., 72., 76., 81.]),
 array([64., 69., 72., 81.]),
 array([64., 69., 81.]),
 array([64., 69.]),
 array([64., 69.]),
 array([64., 69.]),
 array([64., 69.]),
 array([64., 69.]),
 array([64., 69.]),
 array([62., 64., 69.]),
 array([69., 81.]),
 array([64., 69., 76., 81.]),
 array([64., 69., 72., 76., 81., 84.]),
 array([64., 69., 72., 76., 81., 84.]),
 array([64., 69., 72., 74., 76., 81., 84.]),
 array([60., 64., 69., 72., 74., 76., 81., 84.]),
 array([60., 64., 69., 72., 74., 76., 77., 79., 81., 84.]),
 array([60., 64., 69., 72., 74., 76., 77., 79., 81., 84.]),
 array([60., 64., 67., 69., 72., 74., 76., 77., 79., 81., 84.]),
 array([60., 64., 67., 69., 72., 74., 76., 77., 79., 81., 84.]),
 array([60., 64., 67., 69., 72., 74., 76., 77., 79., 81., 84.]),
 array([60., 64., 67., 69., 72., 74., 76., 77., 79

In [27]:
# Перевод .pickle в аккорды и далее в .mid

import mido
import pickle
import numpy as np

timestep_duration = 0.00000005
ticks_per_beat = 80
tempo = 500000
note_duration_ticks = ticks_per_beat

def load_predictions():
    with open('/kaggle/working/Testpred_majority0.3.pickle', 'rb') as f:
        predictions = pickle.load(f)
    return predictions

def tonnetz_to_midi(tonnetz_predictions):
    midi_file = mido.MidiFile()
    track = mido.MidiTrack()
    midi_file.tracks.append(track)

    for timestep, pitch_out_batch in enumerate(tonnetz_predictions):
        time = int(timestep * timestep_duration * ticks_per_beat)
        if len(pitch_out_batch) > 0:
            chord_notes = [int(pitch) for pitch in pitch_out_batch]

            if len(chord_notes) > 0:
                for note in chord_notes:
                    note_on = mido.Message('note_on', note=note, velocity=64, time=time)
                    track.append(note_on)

                for note in chord_notes:
                    note_off = mido.Message('note_off', note=note, velocity=64, time=time + note_duration_ticks)
                    track.append(note_off)
        else:
            pause = mido.Message('note_off', note=0, velocity=0, time=time)
            track.append(pause)

    return midi_file

tonnetz_predictions = load_predictions()
midi_file = tonnetz_to_midi(tonnetz_predictions)
output_path = '/kaggle/working/jazz_output_chords_80.mid'
midi_file.save(output_path)


In [29]:
import mido
import pickle
import numpy as np

timestep_duration = 0.00000005
ticks_per_beat = 80
tempo = 500000
note_duration_ticks = ticks_per_beat

def load_predictions():
    with open('/kaggle/working/Testpred_majority0.3.pickle', 'rb') as f:
        predictions = pickle.load(f)
    return predictions

def remove_repeated_lists(predictions):
    unique_predictions = []
    count = 0
    for prediction in predictions:
        if unique_predictions and np.array_equal(unique_predictions[-1], prediction):
            count += 1
            if count < 3:
                unique_predictions.append(prediction)
        else:
            count = 0
            unique_predictions.append(prediction)
    return unique_predictions

def tonnetz_to_midi(tonnetz_predictions):
    midi_file = mido.MidiFile()
    track = mido.MidiTrack()
    midi_file.tracks.append(track)

    for timestep, pitch_out_batch in enumerate(tonnetz_predictions):
        time = int(timestep * timestep_duration * ticks_per_beat)
        if len(pitch_out_batch) > 0:
            chord_notes = [int(pitch) for pitch in pitch_out_batch]

            if len(chord_notes) > 0:
                for note in chord_notes:
                    note_on = mido.Message('note_on', note=note, velocity=64, time=time)
                    track.append(note_on)

                for note in chord_notes:
                    note_off = mido.Message('note_off', note=note, velocity=64, time=time + note_duration_ticks)
                    track.append(note_off)
        else:
            pause = mido.Message('note_off', note=0, velocity=0, time=time)
            track.append(pause)

    return midi_file

tonnetz_predictions = load_predictions()
tonnetz_predictions_unique = remove_repeated_lists(tonnetz_predictions)
midi_file = tonnetz_to_midi(tonnetz_predictions_unique)
output_path = '/kaggle/working/jazz_output_chords_lessrep_80.mid'
midi_file.save(output_path)


In [46]:
# Варьировала тут настройки

import mido
import pickle
import numpy as np
import subprocess
import os

timestep_duration = 0.00000005
ticks_per_beat = 75 # this one
tempo = 500000
note_duration_ticks = ticks_per_beat

def load_predictions():
    with open('/kaggle/working/Testpred_majority0.3.pickle', 'rb') as f:
        predictions = pickle.load(f)
    return predictions

def remove_repeated_lists(predictions):
    unique_predictions = []
    count = 0
    for prediction in predictions:
        if unique_predictions and np.array_equal(unique_predictions[-1], prediction):
            count += 1
            if count < 2: # this one
                unique_predictions.append(prediction)
        else:
            count = 0
            unique_predictions.append(prediction)
    return unique_predictions

def tonnetz_to_midi(tonnetz_predictions):
    midi_file = mido.MidiFile()
    track = mido.MidiTrack()
    midi_file.tracks.append(track)

    for timestep, pitch_out_batch in enumerate(tonnetz_predictions):
        time = int(timestep * timestep_duration * ticks_per_beat)
        if len(pitch_out_batch) > 0:
            chord_notes = [int(pitch) for pitch in pitch_out_batch]

            if len(chord_notes) > 0:
                for note in chord_notes:
                    note_on = mido.Message('note_on', note=note, velocity=64, time=time)
                    track.append(note_on)

                for note in chord_notes:
                    note_off = mido.Message('note_off', note=note, velocity=64, time=time + note_duration_ticks)
                    track.append(note_off)
        else:
            pause = mido.Message('note_off', note=0, velocity=0, time=time)
            track.append(pause)

    return midi_file
#add some postprocess
def process_midi_with_fluidsynth(midi_path, soundfont_path, output_path):
    cmd = f'fluidsynth -T wav -F "{output_path}" "{soundfont_path}" "{midi_path}"'
    subprocess.run(cmd, shell=True)

# Загрузка предсказаний
tonnetz_predictions = load_predictions()
tonnetz_predictions_unique = remove_repeated_lists(tonnetz_predictions)

# Создание MIDI-файла
midi_file = tonnetz_to_midi(tonnetz_predictions_unique)
output_midi_path = '/kaggle/working/jazz_output_chords_lessrep2FS_75.mid'
midi_file.save(output_midi_path)

# Указание пути к SoundFont файлу
soundfont_path = './font.sf2' 
